In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
display(df)

df.corr()

In [ ]:
from matplotlib import pyplot as plot
import seaborn

df_data = df.drop("Outcome", axis=1)
seaborn.pairplot(df_data, kind="scatter")

In [ ]:
df.describe()

In [ ]:
#gluccose less than 72mg/dl
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn import clone

X = df.drop("Outcome", axis=1)
Y = df["Outcome"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
sgd = SGDClassifier(loss="modified_huber", learning_rate="adaptive", eta0=910, random_state=123)

sdg_clone = clone(sgd)
sdg_clone.fit(X_train, Y_train)

y_pred_sgd = sdg_clone.predict(X_test)

print(y_pred_sgd.shape, Y_test.shape)
print(f"SGD : Precision {precision_score(Y_test, y_pred_sgd)}, Accuracy {accuracy_score(Y_test, y_pred_sgd)}, Recall {precision_score(Y_test, y_pred_sgd)}")


In [ ]:
logistic = LogisticRegression(penalty="l2", solver="liblinear", random_state= 123,)
logistic_clone = clone(logistic)
logistic_clone.fit(X_train, Y_train)

y_pred_logis = logistic_clone.predict(X_test)

print(y_pred_logis.shape, Y_test.shape)
print(f"Logistic : Precision {precision_score(Y_test, y_pred_logis)}, Accuracy {accuracy_score(Y_test, y_pred_logis)}, Recall {precision_score(Y_test, y_pred_logis)}")


In [ ]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

rf = RandomForestClassifier(n_estimators=12, random_state= 123)
tree = DecisionTreeClassifier(random_state=123)

forest = AdaBoostClassifier(base_estimator=clone(rf), algorithm="SAMME",
                            random_state= 123)
forest_clone = clone(forest)
forest_clone.fit(X_train, Y_train)

y_pred_forest = forest_clone.predict(X_test)

print(y_pred_forest.shape, Y_test.shape)
print(f"Ada : Precision {precision_score(Y_test, y_pred_forest)}, Accuracy {accuracy_score(Y_test, y_pred_forest)}, Recall {precision_score(Y_test, y_pred_forest)}")


In [ ]:
import xgboost

xgb_class = xgboost.XGBClassifier(booster='gbtree',
       n_estimators=800, 
       objective='binary:logistic', random_state=123)

xgb_class_clone = clone(xgb_class)

xgb_class_clone.fit(X_train, Y_train)
y_pred_xdb = xgb_class_clone.predict(X_test)
print(y_pred_xdb.shape, Y_test.shape)
print(f"XGB : Precision {precision_score(Y_test, y_pred_xdb)}, Accuracy {accuracy_score(Y_test, y_pred_xdb)}, Recall {precision_score(Y_test, y_pred_xdb)}")


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler 

fill_values = SimpleImputer(missing_values=0, strategy="median")

X_train_imputed = fill_values.fit_transform(X_train)
X_test_imputed = fill_values.transform(X_test)

scaler = StandardScaler()
X_train_imputed = scaler.fit_transform(X_train_imputed)
X_test_imputed = scaler.transform(X_test)

#,                            ("randforest", clone(rf)), ("xgb", clone(xgb_class)), , 
classification = VotingClassifier([ ("logistic", clone(logistic)), ("ada", clone(forest))], voting="soft")

voting_clone = clone(classification)
voting_clone.fit(X_train_imputed, Y_train)
y_pred_vote = voting_clone.predict(X_test_imputed)

print(y_pred_vote.shape, Y_test.shape)
print(f"voting : Precision {precision_score(Y_test, y_pred_vote)}, Accuracy {accuracy_score(Y_test, y_pred_vote)}, Recall {precision_score(Y_test, y_pred_vote)}")


from sklearn.model_selection import cross_val_score
score=cross_val_score(classification,X,Y.ravel(),cv=10)

score.mean()
